In [1]:
# Exercises for Module 6 - API and Python

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import requests

from config import weather_api_key

In [3]:
# Create 1500 random lat & long values
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high = 180.000, size = 1500)
lat_lngs = zip(lats,lngs)
coordinates = list(lat_lngs)  # Creates a list of 1500 tuples
coordinates[:10]

[(-70.61963308939848, 17.07642445635355),
 (7.276434942304618, -49.28629218777408),
 (37.86995246409806, 76.87628240861005),
 (76.84777105942902, -37.36712048357148),
 (-38.701494270922275, -40.69755046346762),
 (43.8681975005135, -32.5815184623668),
 (-22.70980733257042, -155.47630819698975),
 (45.29369037286307, 36.244965537109664),
 (69.19872709519416, 168.74695359784852),
 (78.44722999326109, 99.45154572242859)]

In [4]:
# Explore citipy function with a small sample
mini_sample = coordinates[:5]
mini_sample

[(-70.61963308939848, 17.07642445635355),
 (7.276434942304618, -49.28629218777408),
 (37.86995246409806, 76.87628240861005),
 (76.84777105942902, -37.36712048357148),
 (-38.701494270922275, -40.69755046346762)]

In [5]:
for coordinate in mini_sample:
    print(citipy.nearest_city(coordinate[0], coordinate[1]).city_name,
          citipy.nearest_city(coordinate[0], coordinate[1]).country_code)


bredasdorp za
cayenne gf
shache cn
tasiilaq gl
cidreira br


In [6]:
# Create list for holding cities for the entire example
cities = []
# Find nearest city and add to cities list if new.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0],coordinate[1]).city_name
    if city not in cities:
        cities.append(city)

print(len(cities))

597


In [16]:
# Starting with the Open Weather API stuff (Section 6.2.3)
# Starting URL for Weather Map API Call.

url = "http://api.openweathermap.org/data/2.5/weather?"
city_name = "Boston"
query_url = url + "q="+city_name + "&units=Imperial&APPID=" + weather_api_key



In [17]:
city_weather = requests.get(query_url).json()

In [18]:
city_weather.keys()

dict_keys(['coord', 'weather', 'base', 'main', 'visibility', 'wind', 'rain', 'clouds', 'dt', 'sys', 'timezone', 'id', 'name', 'cod'])

In [20]:
city_weather

{'coord': {'lon': -71.0598, 'lat': 42.3584},
 'weather': [{'id': 501,
   'main': 'Rain',
   'description': 'moderate rain',
   'icon': '10d'}],
 'base': 'stations',
 'main': {'temp': 46.67,
  'feels_like': 43.18,
  'temp_min': 44.29,
  'temp_max': 48.58,
  'pressure': 1020,
  'humidity': 94},
 'visibility': 10000,
 'wind': {'speed': 7, 'deg': 98, 'gust': 20},
 'rain': {'1h': 1.78},
 'clouds': {'all': 100},
 'dt': 1622306310,
 'sys': {'type': 2,
  'id': 2013408,
  'country': 'US',
  'sunrise': 1622279498,
  'sunset': 1622333525},
 'timezone': -14400,
 'id': 4930956,
 'name': 'Boston',
 'cod': 200}

In [21]:
other_call = requests.get(query_url)  # response of 200 indicates the API call got a valid response
other_call

<Response [200]>

In [22]:
# practice retrieving pieces of the JSON data
city_weather.keys()

dict_keys(['coord', 'weather', 'base', 'main', 'visibility', 'wind', 'rain', 'clouds', 'dt', 'sys', 'timezone', 'id', 'name', 'cod'])

In [23]:
city_weather['sys']

{'type': 2,
 'id': 2013408,
 'country': 'US',
 'sunrise': 1622279498,
 'sunset': 1622333525}

In [24]:
city_weather['sys']['country']

'US'

In [25]:
city_weather['coord']

{'lon': -71.0598, 'lat': 42.3584}

In [26]:
city_weather['coord']['lat']

42.3584

In [28]:
city_weather['main']

{'temp': 46.67,
 'feels_like': 43.18,
 'temp_min': 44.29,
 'temp_max': 48.58,
 'pressure': 1020,
 'humidity': 94}

In [29]:
city_weather['main']['temp_max']

48.58

In [30]:
lat = city_weather["coord"]["lat"]
lng = city_weather["coord"]["lon"]
max_temp = city_weather["main"]["temp_max"]
humidity = city_weather["main"]["humidity"]
clouds = city_weather["clouds"]["all"]
wind = city_weather["wind"]["speed"]
print(lat, lng, max_temp, humidity, clouds, wind)

42.3584 -71.0598 48.58 94 100 7


In [31]:
# Convert UTC timestamp
from datetime import datetime


In [33]:
date = city_weather['dt']
date

1622306310

In [35]:
datetime.utcfromtimestamp(date).strftime('%Y-%m-%d %H:%M:%S')

'2021-05-29 16:38:30'

In [ ]:
# Section 6.2.6 - Exercises; continuing with API call for 500 cities
city_data = []
print("----------------")
print("beginning data retrieval")
print("----------------")
record_count = 1
set_count = 1

for i, city in enumerate(cities):
    # Group sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    city_url = url + "&q" + city.replace(" ","+")  # replace blanks with + for string
